# Assignment 2 
### Team Memebers:
 - Bhargav Patel - CS23MTECH11026
 - Manan Patel - CS23MTECH14006
 - Hrishikesh Hemke - CS23MTECH14003
 - Yug Patel - CS23MTECH14019

In [3]:
import numpy as np
from sympy import Matrix
import pandas as pd

In [4]:
class Simplex_Algo():

    filePath = None
    A =None
    b =None
    z =None
    c =None

    def __init__(self,filePath):
        nArr = pd.read_csv(filePath, header=None)

        # Extracting the data
        self.z = nArr.iloc[0, :-1].values.reshape(-1, 1)
        self.c = nArr.iloc[1, :-1].values
        self.b = nArr.iloc[2:, -1:].values
        self.A = nArr.iloc[2:, :-1].values

    # Step 2: function to move towards a vertex z from a given point z
    def find_initial_vertex(self,z):

        A1 = self.A[np.where((np.dot(self.A, z)) == self.b)[0]]
        
        print('Initial vertex Tour:')

        while A1.shape[0] != self.A.shape[1]:

            cost = np.dot(self.c, z)
            
            flat_eles = list(map(float, z.flatten()))
            print(f'z: {tuple((flat_eles))}; cost: {np.round(cost[0], decimals=2)}')

           
            A1 = self.A[np.where((np.dot(self.A, z)) == self.b)[0]]
            A2 = self.A[np.where((np.dot(self.A, z)) != self.b)[0]]
            B2 = self.b[np.where((np.dot(self.A, z)) != self.b)[0]]
            n = A1.shape[0]
            
            null_sp = None
            if A1.shape[0] == 0:
                null_sp = [np.array([[1]]+[[0]]*(len(z)-1))]
            else:
                null_sp = np.array(Matrix(A1).nullspace(), dtype=float)
        
            i = 0
            while A1.shape[0] <= n:
                
                if i > len(null_sp):
                    print("No direction found")
                    break
            
                S = null_sp[i]
                bounded_condition = False
                for i in range(len(A2)):
                    if np.dot(A2[i], S)[0] == 0:
                        continue
                    k = np.round((B2[i] - np.dot(A2[i], z))/np.dot(A2[i], S))
                    if k > 0:
                        bounded_condition = True
            
                if not bounded_condition:
                    print("\nUnbounded  Polytope!!")
                    return None

                for i in range(len(A2)):
                    if np.dot(A2[i], S)[0] == 0:
                        continue
                    k = np.round((B2[i] - np.dot(A2[i], z))/np.dot(A2[i], S))
                    z_temp = z + k*S
                    if all(np.dot(self.A, np.round(z_temp,decimals=2)) <= self.b):
                        z = z_temp
                        break
                    else:
                        continue
                
                cost = np.dot(self.c, z) 
                A1 = self.A[np.where((np.dot(self.A, np.round(z, decimals = 2))) == self.b)[0]]
                A2 = self.A[np.where((np.dot(self.A, z)) != self.b)[0]]
                B2 = self.b[np.where((np.dot(self.A, z)) != self.b)[0]]
                i += 1

        z = np.round(z, decimals = 2)
        
        cost = np.dot(self.c, z)
        flat_eles = list(map(float, z.flatten()))
        print(f'z: {tuple((flat_eles))}; cost: {np.round(cost[0], decimals=2)}')
        return z


    # Step 3: function to move towards an optimal vertex from a non optimal vertex
    def vertex_marching(self,z):

        A1 = self.A[np.where((np.dot(self.A, z)) == self.b)[0]]

        epsilons = np.linalg.inv(A1.T).dot(self.c)

        print("\nJourney to an optimal vertex:")

        while False in (epsilons >= 0):

            cost = np.dot(self.c, z)
            flat_eles = list(map(float, z.flatten()))
            print(f'z: {tuple(flat_eles)}; cost: {np.round(cost[0], decimals=2)}')
            A1inv = np.linalg.inv(A1)
            C = None
            for i in range(len(epsilons)):
                if epsilons[i] < 0:
                    C = A1inv[:, i:i+1]
                    break
            
            if C is None:
                break
            ztemp = z

            A1 = self.A[np.where((np.dot(self.A, np.round(ztemp, decimals=2))) == self.b)[0]]
            A2 = self.A[np.where((np.dot(self.A, np.round(ztemp, decimals=2))) != self.b)[0]]
            B2 = self.b[np.where((np.dot(self.A, np.round(ztemp, decimals=2))) != self.b)[0]]
            
            bounded_condition = False
            for i in range(len(A2)):
                if np.dot(A2[i], C)[0] == 0:
                    continue
                k = np.round((B2[i] - np.dot(A2[i], z))/np.dot(A2[i], C))
                z_temp = ztemp + k*C
                if all(np.dot(self.A, np.round(z_temp,decimals=2)) <= self.b):
                    ztemp = z_temp
                    bounded_condition = True
                    break
                else:
                    continue
                                 
            if not bounded_condition:
                print("\nUnbounded Polytope!!")
                return None

            A1 = self.A[np.where((np.dot(self.A, np.round(ztemp, decimals=2))) == self.b)[0]]
            A2 = self.A[np.where((np.dot(self.A, np.round(ztemp, decimals=2))) != self.b)[0]]
            B2 = self.b[np.where((np.dot(self.A, np.round(ztemp, decimals=2))) != self.b)[0]]
            
            cost = np.dot(self.c, ztemp)
            z = np.round(ztemp, decimals=2)
            epsilons = np.linalg.inv(A1.T).dot(self.c)

        
        z = np.round(z, decimals = 2)
        cost = np.dot(self.c, z)
        flat_eles = list(map(float, z.flatten()))
        print(f'z: {tuple((flat_eles))}; cost: {np.round(cost[0], decimals=2)}')

        return z

    def fit(self):

        point = tuple((map(float, list(self.z.ravel()))))
        # Step 1: Check if the initial point is feasible
        if not (all(np.dot(self.A, self.z) <= self.b)):
            print(f"The initial point z: {point} is not feasible.\n")
            return None
        else:
            print(f"The initial point z: {point} is feasible\n")

        # Step 2: Finding a initial vertex
        vertex = self.find_initial_vertex(self.z)
        
        if vertex is None:
            return None
        # Step 3: From initial vertex move to an optimal vertex
        optimal_vertex = self.vertex_marching(vertex)
        
        return optimal_vertex

In [7]:
algo = Simplex_Algo('Assignment2.csv')
optimal_solution = algo.fit()
if optimal_solution is not None:
    optimal_solution = Matrix(optimal_solution)
    print("\nOptimal cost of the objective function :")
optimal_solution

The initial point z: (0.0, 20.0) is feasible

Initial vertex Tour:
z: (0.0, 20.0); cost: 60

Journey to an optimal vertex:
z: (0.0, 20.0); cost: 60

Unbounded Polytope!!


In [8]:
# Test Input for above Output 

# 0,20,0
# 1,3,0
# -1,1,20
# -2,1,50
# -1,0,0
# 0,-1,0